# Model Monitoring

## Importing Libraries and Setting up Paths

In [2]:
%%time

from datetime import datetime, timedelta, timezone
import json
import os
import re
import boto3
from time import sleep
from threading import Thread

import pandas as pd

from sagemaker import get_execution_role, session, Session, image_uris
from sagemaker.s3 import S3Downloader, S3Uploader
from sagemaker.processing import ProcessingJob
from sagemaker.serializers import CSVSerializer

from sagemaker.model import Model
from sagemaker.model_monitor import DataCaptureConfig

session = Session()

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
CPU times: user 1.23 s, sys: 267 ms, total: 1.5 s
Wall time: 1.5 s


In [16]:
from sagemaker.model_monitor import ModelQualityMonitor
from sagemaker.model_monitor import EndpointInput
from sagemaker.model_monitor.dataset_format import DatasetFormat

In [3]:
# Get Execution role
role = get_execution_role()
print("RoleArn:", role)

region = session.boto_region_name
print("Region:", region)

RoleArn: arn:aws:iam::035349649198:role/LabRole
Region: us-east-1


In [4]:
bucket = session.default_bucket()
print("Churn Bucket:", bucket)
prefix = "sagemaker/InternetChurn-ModelQualityMonitor"

##S3 prefixes
data_capture_prefix = f"{prefix}/datacapture"
s3_capture_upload_path = f"s3://{bucket}/{data_capture_prefix}"

ground_truth_upload_path = (
    f"s3://{bucket}/{prefix}/ground_truth_data/{datetime.now():%Y-%m-%d-%H-%M-%S}"
)

reports_prefix = f"{prefix}/reports"
s3_report_path = f"s3://{bucket}/{reports_prefix}"

##Get the model monitor image
monitor_image_uri = image_uris.retrieve(framework="model-monitor", region=region)

print("Image URI:", monitor_image_uri)
print(f"Capture path: {s3_capture_upload_path}")
print(f"Ground truth path: {ground_truth_upload_path}")
print(f"Report path: {s3_report_path}")

Churn Bucket: sagemaker-us-east-1-035349649198
Image URI: 156813124566.dkr.ecr.us-east-1.amazonaws.com/sagemaker-model-monitor-analyzer
Capture path: s3://sagemaker-us-east-1-035349649198/sagemaker/InternetChurn-ModelQualityMonitor/datacapture
Ground truth path: s3://sagemaker-us-east-1-035349649198/sagemaker/InternetChurn-ModelQualityMonitor/ground_truth_data/2024-02-28-01-50-14
Report path: s3://sagemaker-us-east-1-035349649198/sagemaker/InternetChurn-ModelQualityMonitor/reports


In [5]:
model_url = 's3://sagemaker-us-east-1-035349649198/xgboost-project/xgb_output/xgb-2024-02-23-01-25-43/xgb-2024-02-23-01-25-43/output/model.tar.gz'
model_url

's3://sagemaker-us-east-1-035349649198/xgboost-project/xgb_output/xgb-2024-02-23-01-25-43/xgb-2024-02-23-01-25-43/output/model.tar.gz'

## Deploying Trained XGBoost Model

In [6]:
model_name = f"xgb-churn-pred-model-monitor-{datetime.utcnow():%Y-%m-%d-%H%M}"

# Model version 1.7-1
image_uri = image_uris.retrieve(framework="xgboost", version="1.7-1", region=region)

model = Model(image_uri=image_uri, model_data=model_url, role=role, sagemaker_session=session)

In [7]:
# Creating Endpoint
endpoint_name = f"xgb-churn-model-quality-monitor-{datetime.utcnow():%Y-%m-%d-%H%M}"
print("EndpointName =", endpoint_name)

# Enable Data Capture
data_capture_config = DataCaptureConfig(
    enable_capture=True, sampling_percentage=100, destination_s3_uri=s3_capture_upload_path
)

# Deploy pretrained model
model.deploy(
    initial_instance_count=1,
    instance_type="ml.m5.xlarge",
    endpoint_name=endpoint_name,
    data_capture_config=data_capture_config,
)

EndpointName = xgb-churn-model-quality-monitor-2024-02-28-0150
-----!

In [8]:
from sagemaker.predictor import Predictor
# Creating predictor object from endpoint just invoked
predictor = Predictor(endpoint_name=endpoint_name, sagemaker_session=session, serializer=CSVSerializer())

## Setting up Baseline Dataset

In [9]:
# Choosing a value for cutoff that is considered a customer will churn
churn_cutoff = 0.7
# Creating empty csv to fill with probability and predictions
validate_dataset = "validations_predictions.csv"

In [10]:
# Prof instructed needed 200 samples to compute sd
limit = 200 
i = 0
# Opening validation data
with open(f"test_data/{validate_dataset}", "w") as baseline_file:
    # Writing probability and prediciton
    baseline_file.write("probability,prediction,label\n") 
    with open("test_data/val (5).csv", "r") as f:
        for row in f:
            (label, input_cols) = row.split(",", 1)
            #print(f"Sending data for prediction: {input_cols}")
            #output = predictor.predict(input_cols)
            #print(output)
            # Getting probability with predictor object
            probability = float(predictor.predict(input_cols))
            # Getting prediction with cutoff value
            prediction = "1" if probability > churn_cutoff else "0"
            baseline_file.write(f"{probability},{prediction},{label}\n")
            i += 1
            if i > limit:
                break
            print(".", end="", flush=True)
            sleep(0.5)
print()
print("Done!")

........................................................................................................................................................................................................
Done!


In [11]:
# Ensuring file has populated
!head test_data/validations_predictions.csv

probability,prediction,label
0.9991006851196289,1,1
0.18579556047916412,0,0
0.7986509799957275,1,1
0.012581335380673409,0,0
0.6700054407119751,0,1
0.5484716296195984,0,1
0.025470763444900513,0,0
0.9689635634422302,1,1
0.10012273490428925,0,0


In [14]:
# Uploading predictions using validation set to get model quality
# baseline stats
baseline_prefix = prefix + "/baselining"
baseline_data_prefix = baseline_prefix + "/data"
baseline_results_prefix = baseline_prefix + "/results"
# Setting up path
baseline_data_uri = f"s3://{bucket}/{baseline_data_prefix}"
baseline_results_uri = f"s3://{bucket}/{baseline_results_prefix}"
print(f"Baseline data uri: {baseline_data_uri}")
print(f"Baseline results uri: {baseline_results_uri}")

Baseline data uri: s3://sagemaker-us-east-1-035349649198/sagemaker/InternetChurn-ModelQualityMonitor/baselining/data
Baseline results uri: s3://sagemaker-us-east-1-035349649198/sagemaker/InternetChurn-ModelQualityMonitor/baselining/results


In [15]:
# Uploading baseline
baseline_dataset_uri = S3Uploader.upload(f"test_data/{validate_dataset}", baseline_data_uri)
baseline_dataset_uri

's3://sagemaker-us-east-1-035349649198/sagemaker/InternetChurn-ModelQualityMonitor/baselining/data/validations_predictions.csv'

## Create Baseline Job

In [17]:
# Setting up model monitor object 
churn_model_quality_monitor = ModelQualityMonitor(
    role=role,
    instance_count=1,
    instance_type="ml.m5.xlarge",
    volume_size_in_gb=20,
    max_runtime_in_seconds=1800,
    sagemaker_session=session,)

In [18]:
# Naming the baseline job
baseline_job_name = f"xgb-internet-churn-model-baseline-job-{datetime.utcnow():%Y-%m-%d-%H%M}"

In [19]:
# Execute job, specifying binary classification and prediction/probability columns
job = churn_model_quality_monitor.suggest_baseline(
    job_name=baseline_job_name,
    baseline_dataset=baseline_dataset_uri,
    dataset_format=DatasetFormat.csv(header=True),
    output_s3_uri=baseline_results_uri,
    problem_type="BinaryClassification",
    inference_attribute="prediction",
    probability_attribute="probability",
    ground_truth_attribute="label",
)
job.wait(logs=False)

INFO:sagemaker:Creating processing-job with name xgb-internet-churn-model-baseline-job-2024-02-28-0159


...............................................................................!

## Viewing Baseline Job

In [20]:
# Naming baseline ob
baseline_job = churn_model_quality_monitor.latest_baselining_job

In [21]:
# Viewing metrics
binary_metrics = baseline_job.baseline_statistics().body_dict["binary_classification_metrics"]
pd.json_normalize(binary_metrics).T

,0
confusion_matrix.0.0,87
confusion_matrix.0.1,5
confusion_matrix.1.0,20
confusion_matrix.1.1,89
recall.value,0.816514
recall.standard_deviation,0.020083
precision.value,0.946809
precision.standard_deviation,0.013595
accuracy.value,0.875622
accuracy.standard_deviation,0.011653


In [22]:
# View contraints
pd.DataFrame(baseline_job.suggested_constraints().body_dict["binary_classification_constraints"]).T

,threshold,comparison_operator
recall,0.816514,LessThanThreshold
precision,0.946809,LessThanThreshold
accuracy,0.875622,LessThanThreshold
true_positive_rate,0.816514,LessThanThreshold
true_negative_rate,0.945652,LessThanThreshold
false_positive_rate,0.054348,GreaterThanThreshold
false_negative_rate,0.183486,GreaterThanThreshold
auc,0.945852,LessThanThreshold
f0_5,0.917526,LessThanThreshold
f1,0.876847,LessThanThreshold


## Continuous Model Monitoring for Model Quality Drift

In [26]:
# Method to envoke endpoint designed to send data to a specific SageMaker 
# endpoint for model inference
def invoke_endpoint(ep_name, file_name):
    with open(file_name, "r") as f:
        i = 0
        # Read file line by line
        for row in f:
            # Send each row as payload to endpoint
            payload = row.rstrip("\n")
            response = session.sagemaker_runtime_client.invoke_endpoint(
                EndpointName=endpoint_name,
                ContentType="text/csv",
                Body=payload,
                # Have a unique ID for each row
                # Inference ID used to join prediciton with ground truth
                InferenceId=str(i),
            )["Body"].read()
            i += 1
            sleep(1)
# Method to continuously invoke invoke_endpoint function 
def invoke_endpoint_forever():
    while True:
        try:
            invoke_endpoint(endpoint_name, "test_data/test-Copy1.csv")
        # Igore exception and attempt operator again
        except session.sagemaker_runtime_client.exceptions.ValidationError:
            pass
# Thread initialization and start
thread = Thread(target=invoke_endpoint_forever)
thread.start()

In [27]:
# View captured data
print("Waiting for captures to show up", end="")
for _ in range(120):
    # List files sorted and check if non empty
    # Read last file
    capture_files = sorted(S3Downloader.list(f"{s3_capture_upload_path}/{endpoint_name}"))
    if capture_files:
        capture_file = S3Downloader.read_file(capture_files[-1]).split("\n")
        capture_record = json.loads(capture_file[0])
        # Verify file is a capture file with inference data
        if "inferenceId" in capture_record["eventMetadata"]:
            break
    print(".", end="", flush=True)
    sleep(1)
print()
# Show last three captured files
print("Found Capture Files:")
print("\n ".join(capture_files[-3:]))

Waiting for captures to show up............................
Found Capture Files:
s3://sagemaker-us-east-1-035349649198/sagemaker/InternetChurn-ModelQualityMonitor/datacapture/xgb-churn-model-quality-monitor-2024-02-28-0150/AllTraffic/2024/02/28/01/53-17-119-e26899d0-195c-408e-8577-727ead6bae21.jsonl
 s3://sagemaker-us-east-1-035349649198/sagemaker/InternetChurn-ModelQualityMonitor/datacapture/xgb-churn-model-quality-monitor-2024-02-28-0150/AllTraffic/2024/02/28/01/54-17-127-89c9930e-a4a0-48fc-bddb-96e708811f7b.jsonl
 s3://sagemaker-us-east-1-035349649198/sagemaker/InternetChurn-ModelQualityMonitor/datacapture/xgb-churn-model-quality-monitor-2024-02-28-0150/AllTraffic/2024/02/28/02/16-27-824-1b07b328-fcd5-4c91-9600-bb44b1eae14f.jsonl


In [28]:
# View contents of a captured file
print("\n".join(capture_file[-3:-1]))

{"captureData":{"endpointInput":{"observedContentType":"text/csv","mode":"INPUT","data":"-1.0708326375420545,-0.8182815190179041,-0.34415002748619744,0.23343086069704833,-0.14826620909867583,-0.19424909784146976,1.0,1.0,0.0,1.0,0.0,0.0","encoding":"CSV"},"endpointOutput":{"observedContentType":"text/csv; charset=utf-8","mode":"OUTPUT","data":"0.036486055701971054\n","encoding":"CSV"}},"eventMetadata":{"eventId":"2a43499b-0502-4b4f-bd02-14d8a374a075","inferenceId":"57","inferenceTime":"2024-02-28T02:17:25Z"},"eventVersion":"0"}
{"captureData":{"endpointInput":{"observedContentType":"text/csv","mode":"INPUT","data":"-0.887683264183178,0.14277014727270668,-0.34415002748619744,-0.7882289115836785,-0.45097682232168035,-0.19424909784146976,0.0,0.0,1.0,0.0,0.0,0.0","encoding":"CSV"},"endpointOutput":{"observedContentType":"text/csv; charset=utf-8","mode":"OUTPUT","data":"0.9995995163917542\n","encoding":"CSV"}},"eventMetadata":{"eventId":"51e6c4f5-6066-4d70-8329-b9384ce7ed7e","inferenceId":"5

In [29]:
# Take Python dictionary of captured record and convert to JSON
# formatted string
print(json.dumps(capture_record, indent=2))

{
  "captureData": {
    "endpointInput": {
      "observedContentType": "text/csv",
      "mode": "INPUT",
      "data": "1.577408301566023,1.2516759160695654,-0.34415002748619744,-0.7882289115836785,-0.45097682232168035,-0.19424909784146976,1.0,0.0,1.0,0.0,0.0,0.0",
      "encoding": "CSV"
    },
    "endpointOutput": {
      "observedContentType": "text/csv; charset=utf-8",
      "mode": "OUTPUT",
      "data": "0.9955226182937622\n",
      "encoding": "CSV"
    }
  },
  "eventMetadata": {
    "eventId": "de335f6c-e1c6-4401-93b2-7be7c531d0ac",
    "inferenceId": "0",
    "inferenceTime": "2024-02-28T02:16:27Z"
  },
  "eventVersion": "0"
}


## Generating Synthetic Ground Truth

In [30]:
import random
# Function to generate ground truth data for predictions
def ground_truth_with_id(inference_id):
    # Use this to get consistent results
    random.seed(inference_id)  
    rand = random.random()
    # Return dictionary to represent event data and ground truth data
    return {
        "groundTruthData": {
            # Generate random positive labels for 70% of the time
            "data": "1" if rand < 0.7 else "0",
            "encoding": "CSV",
        },
        "eventMetadata": {
            "eventId": str(inference_id),
        },
        "eventVersion": "0",
    }
# Function to upload ground truth records to S3
def upload_ground_truth(records, upload_time):
    # Serialize each record due to S3 needing a string format
    fake_records = [json.dumps(r) for r in records]
    data_to_upload = "\n".join(fake_records)
    # Constructing S3 URI 
    target_s3_uri = f"{ground_truth_upload_path}/{upload_time:%Y/%m/%d/%H/%M%S}.jsonl"
    print(f"Uploading {len(fake_records)} records to", target_s3_uri)
    # Upload to S3
    S3Uploader.upload_string_as_file_body(data_to_upload, target_s3_uri)

In [31]:
# Set ground truth records, rows sending in for inference
# Used professor's count
NUM_GROUND_TRUTH_RECORDS = 334  
# Function for continuous generation and uploading process
def generate_fake_ground_truth_forever():
    j = 0
    while True:
        # Generate fake records to generate a consistent ground truth data
        # record for each inference
        fake_records = [ground_truth_with_id(i) for i in range(NUM_GROUND_TRUTH_RECORDS)]
        # Upload fake records to S3
        upload_ground_truth(fake_records, datetime.utcnow())
        # Wait and repeat
        j = (j + 1) % 5
        sleep(60 * 60)  # do this once an hour
# Threading for function to execute in the background
gt_thread = Thread(target=generate_fake_ground_truth_forever)
gt_thread.start()

Uploading 334 records to s3://sagemaker-us-east-1-035349649198/sagemaker/InternetChurn-ModelQualityMonitor/ground_truth_data/2024-02-28-01-50-14/2024/02/28/02/2019.jsonl


## Create Monitoring Schedule

In [32]:
# Creating monitoring schedule 
churn_monitor_schedule_name = (
    f"xgb-internet-churn-monitoring-schedule-{datetime.utcnow():%Y-%m-%d-%H%M}")

In [33]:
# Set up endpoint input
endpointInput = EndpointInput(
    endpoint_name=predictor.endpoint_name,
    # First element of output
    probability_attribute="0",
    # Threshold for determining cut off for classifying the inference
    probability_threshold_attribute=0.5,
    # S3 destination
    destination="/opt/ml/processing/input_data",)

In [34]:
# Monitoring schedule that executes every hour
from sagemaker.model_monitor import CronExpressionGenerator
# Get response from monitor 
response = churn_model_quality_monitor.create_monitoring_schedule(
    monitor_schedule_name=churn_monitor_schedule_name,
    endpoint_input=endpointInput,
    output_s3_uri=baseline_results_uri,
    problem_type="BinaryClassification",
    ground_truth_input=ground_truth_upload_path,
    constraints=baseline_job.suggested_constraints(),
    # How frequently the monitoring job should run, enable Cloudwatch as well
    schedule_cron_expression=CronExpressionGenerator.hourly(),
    enable_cloudwatch_metrics=True,)

INFO:sagemaker.model_monitor.model_monitoring:Creating Monitoring Schedule with name: xgb-internet-churn-monitoring-schedule-2024-02-28-0221


In [35]:
# Show schedule
churn_model_quality_monitor.describe_schedule()

{'MonitoringScheduleArn': 'arn:aws:sagemaker:us-east-1:035349649198:monitoring-schedule/xgb-internet-churn-monitoring-schedule-2024-02-28-0221',
 'MonitoringScheduleName': 'xgb-internet-churn-monitoring-schedule-2024-02-28-0221',
 'MonitoringScheduleStatus': 'Scheduled',
 'MonitoringType': 'ModelQuality',
 'CreationTime': datetime.datetime(2024, 2, 28, 2, 21, 31, 512000, tzinfo=tzlocal()),
 'LastModifiedTime': datetime.datetime(2024, 2, 28, 2, 21, 42, 493000, tzinfo=tzlocal()),
 'MonitoringScheduleConfig': {'ScheduleConfig': {'ScheduleExpression': 'cron(0 * ? * * *)'},
  'MonitoringJobDefinitionName': 'model-quality-job-definition-2024-02-28-02-21-30-962',
  'MonitoringType': 'ModelQuality'},
 'EndpointName': 'xgb-churn-model-quality-monitor-2024-02-28-0150',
 'ResponseMetadata': {'RequestId': 'b112e384-9c7b-4995-bc70-04f7d33614a6',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'b112e384-9c7b-4995-bc70-04f7d33614a6',
   'content-type': 'application/x-amz-json-1.1',
   

In [36]:
# Note to team: Wait to run until a little bit before top of the hour
executions = churn_model_quality_monitor.list_executions()
executions

[]

In [37]:
# Wait for first execution
print("Waiting for first execution", end="")
while True:
    execution = churn_model_quality_monitor.describe_schedule().get(
        "LastMonitoringExecutionSummary"
    )
    if execution:
        break
    print(".", end="", flush=True)
    sleep(10)
print()
# Let team know that an execution is found
print("Execution found!")

Waiting for first execution.....................................................................................................................................................................................................................................................................
Execution found!


In [39]:
while not executions:
    executions = churn_model_quality_monitor.list_executions()
    print(".", end="", flush=True)
    sleep(10)
latest_execution = executions[-1]
# Show most recent execution
latest_execution.describe()

....................

{'ProcessingInputs': [{'InputName': 'groundtruth_input_1',
   'AppManaged': False,
   'S3Input': {'S3Uri': 's3://sagemaker-us-east-1-035349649198/sagemaker/InternetChurn-ModelQualityMonitor/ground_truth_data/2024-02-28-01-50-14/2024/02/28/02',
    'LocalPath': '/opt/ml/processing/groundtruth/2024/02/28/02',
    'S3DataType': 'S3Prefix',
    'S3InputMode': 'File',
    'S3DataDistributionType': 'FullyReplicated',
    'S3CompressionType': 'None'}},
  {'InputName': 'endpoint_input_1',
   'AppManaged': False,
   'S3Input': {'S3Uri': 's3://sagemaker-us-east-1-035349649198/sagemaker/InternetChurn-ModelQualityMonitor/datacapture/xgb-churn-model-quality-monitor-2024-02-28-0150/AllTraffic/2024/02/28/02',
    'LocalPath': '/opt/ml/processing/input_data/xgb-churn-model-quality-monitor-2024-02-28-0150/AllTraffic/2024/02/28/02',
    'S3DataType': 'S3Prefix',
    'S3InputMode': 'File',
    'S3DataDistributionType': 'FullyReplicated',
    'S3CompressionType': 'None'}}],
 'ProcessingOutputConfig': {'Ou

In [40]:
# Tells team state of execution and status
status = execution["MonitoringExecutionStatus"]

while status in ["Pending", "InProgress"]:
    print("Waiting for execution to finish", end="")
    latest_execution.wait(logs=False)
    latest_job = latest_execution.describe()
    print()
    print(f"{latest_job['ProcessingJobName']} job status:", latest_job["ProcessingJobStatus"])
    print(
        f"{latest_job['ProcessingJobName']} job exit message, if any:",
        latest_job.get("ExitMessage"),
    )
    print(
        f"{latest_job['ProcessingJobName']} job failure reason, if any:",
        latest_job.get("FailureReason"),
    )
    sleep(
        30
    )  
    # Wait for second execution to begin
    latest_execution = churn_model_quality_monitor.list_executions()[-1]
    execution = churn_model_quality_monitor.describe_schedule()["LastMonitoringExecutionSummary"]
    status = execution["MonitoringExecutionStatus"]

print("Execution status is:", status)

if status != "Completed":
    print(execution)
    print(
        "====STOP==== \n No completed executions to inspect further. Please wait till an execution completes or investigate previously reported failures."
    )

Waiting for execution to finish.................................................................!
groundtruth-merge-202402280300-5ca3981c669ab10d4e4675c3 job status: Completed
groundtruth-merge-202402280300-5ca3981c669ab10d4e4675c3 job exit message, if any: None
groundtruth-merge-202402280300-5ca3981c669ab10d4e4675c3 job failure reason, if any: None
Waiting for execution to finish................................................Uploading 334 records to s3://sagemaker-us-east-1-035349649198/sagemaker/InternetChurn-ModelQualityMonitor/ground_truth_data/2024-02-28-01-50-14/2024/02/28/03/2020.jsonl
..........................!
model-quality-monitoring-202402280300-5ca3981c669ab10d4e4675c3 job status: Completed
model-quality-monitoring-202402280300-5ca3981c669ab10d4e4675c3 job exit message, if any: CompletedWithViolations: Job completed successfully with 10 violations.
model-quality-monitoring-202402280300-5ca3981c669ab10d4e4675c3 job failure reason, if any: None
Execution status is: Complete

In [41]:
# Show location of the report
latest_execution = churn_model_quality_monitor.list_executions()[-1]
report_uri = latest_execution.describe()["ProcessingOutputConfig"]["Outputs"][0]["S3Output"][
    "S3Uri"
]
print("Report Uri:", report_uri)

Report Uri: s3://sagemaker-us-east-1-035349649198/sagemaker/InternetChurn-ModelQualityMonitor/baselining/results/xgb-churn-model-quality-monitor-2024-02-28-0150/xgb-internet-churn-monitoring-schedule-2024-02-28-0221/2024/02/28/03


In [42]:
# View violations made by monitoring schedule
pd.options.display.max_colwidth = None
violations = latest_execution.constraint_violations().body_dict["violations"]
violations_df = pd.json_normalize(violations)
violations_df.head(10)

,constraint_check_type,description,metric_name
0,LessThanThreshold,Metric auc with 0.5478350700682718 +/- 0.015239289398642797 was LessThanThreshold '0.9458516154766653',auc
1,LessThanThreshold,Metric precision with 0.7653631284916201 +/- 0.015270305278741994 was LessThanThreshold '0.9468085106382979',precision
2,LessThanThreshold,Metric truePositiveRate with 0.5661157024793388 +/- 0.009778008244012105 was LessThanThreshold '0.8165137614678899',truePositiveRate
3,LessThanThreshold,Metric f1 with 0.65083135391924 +/- 0.010181275767075525 was LessThanThreshold '0.8768472906403939',f1
4,LessThanThreshold,Metric accuracy with 0.5598802395209581 +/- 0.010089283287949334 was LessThanThreshold '0.8756218905472637',accuracy
5,GreaterThanThreshold,Metric falsePositiveRate with 0.45652173913043476 +/- 0.013350980170514427 was GreaterThanThreshold '0.05434782608695652',falsePositiveRate
6,LessThanThreshold,Metric trueNegativeRate with 0.5434782608695652 +/- 0.013350980170514467 was LessThanThreshold '0.9456521739130435',trueNegativeRate
7,GreaterThanThreshold,Metric falseNegativeRate with 0.43388429752066116 +/- 0.009778008244012088 was GreaterThanThreshold '0.1834862385321101',falseNegativeRate
8,LessThanThreshold,Metric recall with 0.5661157024793388 +/- 0.009778008244012105 was LessThanThreshold '0.8165137614678899',recall
9,LessThanThreshold,Metric f2 with 0.5972101133391456 +/- 0.009672410517424079 was LessThanThreshold '0.839622641509434',f2


## View Model Quality CloudWatch Metrics

In [43]:
# Make a CW client 
cw_client = boto3.Session().client("cloudwatch")
namespace = "aws/sagemaker/Endpoints/model-metrics"
cw_dimensions = [
    {"Name": "Endpoint", "Value": endpoint_name},
    {"Name": "MonitoringSchedule", "Value": churn_monitor_schedule_name},]

In [44]:
# Show CW metrics 
paginator = cw_client.get_paginator("list_metrics")

for response in paginator.paginate(Dimensions=cw_dimensions, Namespace=namespace):
    model_quality_metrics = response["Metrics"]
    for metric in model_quality_metrics:
        print(metric["MetricName"])

recall_best_constant_classifier
au_prc
true_negative_rate
f0_5
false_negative_rate
f2_best_constant_classifier
f1
precision_best_constant_classifier
false_positive_rate
precision
total_number_of_violations
f1_best_constant_classifier
true_positive_rate
accuracy_best_constant_classifier
f0_5_best_constant_classifier
recall
auc
f2
accuracy


In [45]:
# Create CW alarm for F2 score
alarm_name = "MODEL_QUALITY_F2_SCORE"
alarm_desc = (
    "Trigger an CloudWatch alarm when the f2 score drifts away from the baseline constraints"
)
mdoel_quality_f2_drift_threshold = (
    0.625  ##Setting this threshold purposefully low to see the alarm quickly.
)
metric_name = "f2"
namespace = "aws/sagemaker/Endpoints/model-metrics"

cw_client.put_metric_alarm(
    AlarmName=alarm_name,
    AlarmDescription=alarm_desc,
    ActionsEnabled=True,
    MetricName=metric_name,
    Namespace=namespace,
    Statistic="Average",
    Dimensions=[
        {"Name": "Endpoint", "Value": endpoint_name},
        {"Name": "MonitoringSchedule", "Value": churn_monitor_schedule_name},
    ],
    Period=600,
    EvaluationPeriods=1,
    DatapointsToAlarm=1,
    Threshold=mdoel_quality_f2_drift_threshold,
    ComparisonOperator="LessThanOrEqualToThreshold",
    TreatMissingData="breaching",
)

{'ResponseMetadata': {'RequestId': '7e70d87b-8bd5-4e51-b781-28a7a3760ea8',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '7e70d87b-8bd5-4e51-b781-28a7a3760ea8',
   'content-type': 'text/xml',
   'content-length': '214',
   'date': 'Wed, 28 Feb 2024 03:23:05 GMT'},
  'RetryAttempts': 0}}

In [46]:
# Delete monitoring schedule
churn_model_quality_monitor.delete_monitoring_schedule()
sleep(60) 

INFO:sagemaker:Deleting Monitoring Schedule with name: xgb-internet-churn-monitoring-schedule-2024-02-28-0221
INFO:sagemaker.model_monitor.model_monitoring:Deleting Model Quality Job Definition with name: model-quality-job-definition-2024-02-28-02-21-30-962


In [48]:
!aws sagemaker list-endpoints

{
    "Endpoints": [
        {
            "EndpointName": "xgb-churn-model-quality-monitor-2024-02-28-0150",
            "EndpointArn": "arn:aws:sagemaker:us-east-1:035349649198:endpoint/xgb-churn-model-quality-monitor-2024-02-28-0150",
            "CreationTime": 1709085016.161,
            "LastModifiedTime": 1709085193.775,
            "EndpointStatus": "InService"
        }
    ]
}


In [49]:
sagemaker = boto3.client('sagemaker')
sagemaker.delete_endpoint(EndpointName="xgb-churn-model-quality-monitor-2024-02-28-0150")

{'ResponseMetadata': {'RequestId': '29e3ce86-53f0-4b78-a0cf-545d28c51ae2',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '29e3ce86-53f0-4b78-a0cf-545d28c51ae2',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '0',
   'date': 'Wed, 28 Feb 2024 03:35:25 GMT'},
  'RetryAttempts': 0}}